In [43]:
import pubchempy as pcp

def get_smiles_pubchempy(drugbank_id):
    compounds = pcp.get_compounds(drugbank_id, 'drugbank')
    if not compounds:
        return None
    return compounds[0].canonical_smiles

print(get_smiles_pubchempy("DB01050"))

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)>

In [44]:
import requests
from requests import Session
from functools import lru_cache
from typing import Optional

BASE_URL = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug'
_session = Session()

@lru_cache(maxsize=128)
def get_smiles_pubchem(drugbank_id: str, timeout: float = 5.0) -> Optional[str]:
    """
    Retrieve the Canonical SMILES for a given DrugBank ID via PubChem PUG REST.
    Returns None on any error or if no SMILES is found.
    """
    try:
        # 1) fetch the substance record and extract CID
        resp = _session.get(
            f'{BASE_URL}/substance/sourceid/drugbank/{drugbank_id}/JSON',
            timeout=timeout
        )
        resp.raise_for_status()
        subs = resp.json().get('PC_Substances', [])
        if not subs:
            return None

        # find the compound entry whose id.type == 1 and id.id.cid exists
        cid = next(
            (
                comp['id']['id']['cid']
                for comp in subs[0].get('compound', [])
                if comp.get('id', {}).get('type') == 1
                   and isinstance(comp['id']['id'].get('cid'), int)
            ),
            None
        )
        if cid is None:
            return None

        # 2) fetch the canonical SMILES for that CID
        resp = _session.get(
            f'{BASE_URL}/compound/cid/{cid}/property/CanonicalSMILES/JSON',
            timeout=timeout
        )
        resp.raise_for_status()
        props = resp.json().get('PropertyTable', {}).get('Properties', [])
        if not props:
            return None

        return props[0].get('CanonicalSMILES')

    except requests.RequestException as e:
        # you could log this instead of printing in production
        print(f"PubChem request failed: {e}")
        return None

In [45]:
print(get_smiles_pubchem("DB01030"))

CCC1(C2=C(COC1=O)C(=O)N3CC4=CC5=C(C=CC(=C5CN(C)C)O)N=C4C3=C2)O


In [63]:
import requests
from requests import Session
from functools import lru_cache
from typing import Optional, List

BASE_URL = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug'
_session = Session()

@lru_cache(maxsize=128)
def get_trade_name_from_smiles(
    smiles: str,
    timeout: float = 5.0
) -> Optional[str]:
    """
    Sucht per SMILES zuerst die PubChem-CID, dann alle Synonyme und
    wählt daraus einen wahrscheinlichen Handelsnamen (erstes Eintrag ohne
    Ziffern/Bindestriche, beginnend mit Großbuchstaben).
    Gibt None zurück, wenn nichts gefunden oder bei Fehlern.
    """
    try:
        # 1) CID per SMILES holen
        r1 = _session.get(
            f'{BASE_URL}/compound/smiles/{smiles}/cids/JSON',
            timeout=timeout
        )
        r1.raise_for_status()
        cids = r1.json().get('IdentifierList', {}).get('CID', [])
        if not cids:
            return None
        cid = cids[0]

        # 2) Synonyme für diesen CID holen
        r2 = _session.get(
            f'{BASE_URL}/compound/cid/{cid}/synonyms/JSON',
            timeout=timeout
        )
        r2.raise_for_status()
        info = r2.json().get('InformationList', {}).get('Information', [])
        #print(info)
        if not info:
            return None
        synonyms: List[str] = info[0].get('Synonym', [])
        return synonyms[0]

    except requests.RequestException as e:
        # In Produktiv-Code lieber logging statt print()
        print(f"Fehler bei PubChem-Abfrage: {e}")
        return None